In [202]:

# foldername = "Data/BGE/selected_10_feeders/Feeder_7130/"
# feedername = "7130"

# foldername = "Data/BGE/selected_10_feeders/Feeder_7361/"
# feedername = "7361"

# foldername = "Data/BGE/selected_10_feeders/Feeder_7366/"
# feedername = "7366"

# foldername = "Data/BGE/selected_10_feeders/Feeder_7425/"
# feedername = "7425"

# foldername = "Data/BGE/selected_10_feeders/Feeder_7616/"
# feedername = "7616"

# foldername = "Data/BGE/selected_10_feeders/Feeder_7651/"
# feedername = "7651"

# foldername = "Data/BGE/selected_10_feeders/Feeder_7731/"
# feedername = "7731"

# foldername = "Data/BGE/selected_10_feeders/Feeder_7779/"
# feedername = "7779"

# foldername = "Data/BGE/selected_10_feeders/Feeder_7873/"
# feedername = "7873"


foldername = "Data/PHI_0217/Feeder_00496-twining_city/"
feedername = "Feeder_00496-twining_city"

# foldername = "Data/PHI_0217/Feeder_14187-livingston/"
# feedername = "Feeder_14187-livingston"

# foldername = "Data/PHI_0217/Feeder_14711-14711/"
# feedername = "Feeder_14711-14711"

# foldername = "Data/PHI_0217/Feeder_14748-sligo/"
# feedername = "Feeder_14748-sligo"

# foldername = "Data/PHI_0217/Feeder_14859-rockville/"
# feedername = "Feeder_14859-rockville"

# foldername = "Data/PHI_0217/Feeder_14884-middlebrook/"
# feedername = "Feeder_14884-middlebrook"

# foldername = "Data/PHI_0217/Feeder_14941-parklawn_drive/"
# feedername = "Feeder_14941-parklawn_drive"

# foldername = "Data/PHI_0217/Feeder_15144-bethesda/"
# feedername = "Feeder_15144-bethesda"

# foldername = "Data/PHI_0217/Feeder_15701-15701/"
# feedername = "Feeder_15701-15701"

# foldername = "Data/PHI_0217/Feeder_15708-15708/"
# feedername = "Feeder_15708-15708"

# PHI CRS
EPSG = '4438'

# BGE CRS
# EPSG = '2804'

with open(foldername + 'Lines.dss') as f:
    lines = f.readlines()

with open(foldername + 'Loads.dss') as f:
    loads = f.readlines()

with open(foldername + 'Transformers.dss') as f:
    trafo = f.readlines()


with open(foldername+'Capacitors.dss') as f:
    caps = f.readlines()

with open(foldername + 'Buscoords.dss') as f:
    buscoord = f.readlines()
    

# Line buses
line_se = []
line_re = []

for i in range(0, len(lines), 2):
    stri = lines[i]

    if stri[4:8] == 'Line':
        idx1 = stri.index('bus1')  # index of bus 1
        idx2 = stri.index('bus2')  # index of bus 2
        idx3 = stri.index('switch')  # index of switch
        bus1_name = stri[idx1 + 5:idx2 - 1]
        bus2_name = stri[idx2 + 5:idx3 - 1]
        line_se.append(bus1_name)
        line_re.append(bus2_name)
        
        
        
# Tranformer buses
trafo_se = []
trafo_re = []

for i in range(0, len(trafo), 2):
    if trafo[i][0:25] == 'New Transformer.trans_Reg': #Regulator branches
        stri = trafo[i]
        idx1 = stri.index('buses') #index of bus 1
        idx2 = stri.index('(', idx1+1) #index of bus 2
        idx3 = stri.index(',', idx2+1) #index of winding
        idx4 = stri.index(')', idx3+1)  # index of winding
        bus1_name = stri[idx2+1:idx3-2]
        bus2_name = stri[idx3+1:idx4-2]
        trafo_se.append(bus1_name)
        trafo_se.append(bus2_name)
        
    else: 
        stri = trafo[i]
        idx1 = stri.index('bus') #index of bus 1
        idx2 = stri.index('bus', idx1+1) #index of bus 2
        idx3 = stri.index('wdg=2') #index of winding
        idx4 = stri.index('XHL')  # index of winding
        bus1_name = stri[idx1+4:idx3-1]
        bus2_name = stri[idx2+4:idx4-1]
        trafo_se.append(bus1_name)
        trafo_se.append(bus2_name)
    
    

    # Load buses
load_bus_list = []
for i in range(0, len(loads), 2):
    stri = loads[i]
    idx1 = stri.index('bus1')  # index of bus 1
    idx2 = stri.index(' ', idx1+1)
    load_bus = stri[idx1 + 5:idx2 - 2]
    load_bus_list.append(load_bus)
    

    # Cap buses
cap_bus_list = []
for i in range(0, len(caps), 2):
    stri = caps[i]
    idx1 = stri.index('Bus1')  # index of bus 1
    idx2 = stri.index('phases')
    cap_bus = stri[idx1 + 5:idx2 - 1]
    cap_bus_list.append(cap_bus)
    
    

bus_list = line_se + line_re + trafo_se + trafo_re + load_bus_list + cap_bus_list

# Bus coordinates
bus_coord_list = []
for i in range(0, len(buscoord), 1):
    stri = buscoord[i]
    idx1 = stri.index(' ')  # index of bus 1
    bus_coord = stri[0:idx1]
    bus_coord_list.append(bus_coord)

In [203]:
# Eliminating phasing (.1.2.3) from bus list to make it comparable to bus coordinates files
new_bus_list = []

for s in bus_list:
    length = len(s)

    if s[length - 12:] == '.1.2.3.1.2.3':
        n = length - 1
        replacement = ''
        res_str = s[0:n - 11] + replacement
        new_bus_list.append(res_str)

    elif s[length - 12:] == '.1.2.3.3.1.2':
        n = length - 1
        replacement = ''
        res_str = s[0:n - 11] + replacement
        new_bus_list.append(res_str)

    elif s[length - 12:] == '.1.2.3.2.1.3':
        n = length - 1
        replacement = ''
        res_str = s[0:n - 11] + replacement
        new_bus_list.append(res_str)

    elif s[length - 12:] == '.3.1.2.1.2.3':
        n = length - 1
        replacement = ''
        res_str = s[0:n - 11] + replacement
        new_bus_list.append(res_str)

    elif s[length - 12:] == '.2.1.3.1.2.3':
        n = length - 1
        replacement = ''
        res_str = s[0:n - 11] + replacement
        new_bus_list.append(res_str)

    elif s[length - 6:] == '.1.2.3':
        n = length - 1
        replacement = ''
        res_str = s[0:n - 5] + replacement
        new_bus_list.append(res_str)

    elif s[length - 6:] == '.1.3.2':
        n = length - 1
        replacement = ''
        res_str = s[0:n - 5] + replacement
        new_bus_list.append(res_str)

    elif s[length - 6:] == '.3.2.1':
        n = length - 1
        replacement = ''
        res_str = s[0:n - 5] + replacement
        new_bus_list.append(res_str)

    elif s[length - 6:] == '.3.1.2':
        n = length - 1
        replacement = ''
        res_str = s[0:n - 5] + replacement
        new_bus_list.append(res_str)


    elif s[length - 6:] == '.2.3.1':
        n = length - 1
        replacement = ''
        res_str = s[0:n - 5] + replacement
        new_bus_list.append(res_str)

    elif s[length - 6:] == '.2.1.3':
        n = length - 1
        replacement = ''
        res_str = s[0:n - 5] + replacement
        new_bus_list.append(res_str)

    elif s[length - 4:] == '.1.1':
        n = length - 1
        replacement = ''
        res_str = s[0:n - 3] + replacement
        new_bus_list.append(res_str)

    elif s[length - 4:] == '.1.3':
        n = length - 1
        replacement = ''
        res_str = s[0:n - 3] + replacement
        new_bus_list.append(res_str)

    elif s[length - 4:] == '.1.2':
        n = length - 1
        replacement = ''
        res_str = s[0:n - 3] + replacement
        new_bus_list.append(res_str)

    elif s[length - 4:] == '.2.1':
        n = length - 1
        replacement = ''
        res_str = s[0:n - 3] + replacement
        new_bus_list.append(res_str)

    elif s[length - 4:] == '.2.2':
        n = length - 1
        replacement = ''
        res_str = s[0:n - 3] + replacement
        new_bus_list.append(res_str)

    elif s[length - 4:] == '.2.3':
        n = length - 1
        replacement = ''
        res_str = s[0:n - 3] + replacement
        new_bus_list.append(res_str)

    elif s[length - 4:] == '.3.1':
        n = length - 1
        replacement = ''
        res_str = s[0:n - 3] + replacement
        new_bus_list.append(res_str)

    elif s[length - 4:] == '.3.2':
        n = length - 1
        replacement = ''
        res_str = s[0:n - 3] + replacement
        new_bus_list.append(res_str)

    elif s[length - 4:] == '.3.3':
        n = length - 1
        replacement = ''
        res_str = s[0:n - 3] + replacement
        new_bus_list.append(res_str)

    elif s[length - 2:] == '.1':
        n = length - 1
        replacement = ''
        res_str = s[0:n - 1] + replacement
        new_bus_list.append(res_str)

    elif s[length - 2:] == '.2':
        n = length - 1
        replacement = ''
        res_str = s[0:n - 1] + replacement
        new_bus_list.append(res_str)

    elif s[length - 2:] == '.3':
        n = length - 1
        replacement = ''
        res_str = s[0:n - 1] + replacement
        new_bus_list.append(res_str)

    else:
        new_bus_list.append(s)

# # First convert buscoords list to uppercase
upper_bus_coords = []
for s in bus_coord_list:
    caps = s.upper()
    # print(f'upper case is written as {caps}')
    upper_bus_coords.append(caps)

# convert bus list to uppercase
upper_bus_list = []
for s in new_bus_list:
    caps = s.upper()
    # print(f'upper case is written as {caps}')
    upper_bus_list.append(caps)

real_bus_set = set(upper_bus_list)  # unique bus list in uppercase
real_bus_list = list(real_bus_set)


# Checking which buses do not have coordinates
res = [x for x in real_bus_list + upper_bus_coords if x not in upper_bus_coords]

# Checking redundant buses which have buscoords
redundant = [x for x in real_bus_list + upper_bus_coords if x not in real_bus_list]


In [204]:
import pyproj

real_bus_coords = []
for r in range(0, len(real_bus_list)):
    if real_bus_list[r] in upper_bus_coords:
        temp = upper_bus_coords.index(real_bus_list[r])
        real_bus_coords.append(buscoord[temp])
        
        
        
bus_list = []
x_list = []
y_list = []
lat_list = []
lon_list = []
buscoord_lon_lat = []
for i in range(0, len(real_bus_coords), 1):
    stri = real_bus_coords[i]
    idx1 = stri.index(' ')
    idx2 = stri.index(' ', idx1 + 1)
    x_temp = float(stri[idx1+1:idx2])
    y_temp = float(stri[idx2 + 1:len(stri)-1])
    x_list.append(x_temp)
    y_list.append(y_temp)
    # BGE Maryland
    #transformer = pyproj.Transformer.from_crs("epsg:2804", "epsg:4326", always_xy=True)
    # PHI Maryland
    #transformer = pyproj.Transformer.from_crs("epsg:3748", "epsg:4326", always_xy=True)
    transformer = pyproj.Transformer.from_crs("epsg:"+EPSG, "epsg:4326", always_xy=True)


    # https://www.spatialreference.org/ref/?search=2804&srtext=Search
    # I used this to look up the projection with reference to Maryland coordinate system
    x2, y2 = transformer.transform(x_temp, y_temp)
    bus_list.append(stri[0:idx1])
    lon_list.append(x2)
    lat_list.append(y2)
    buscoord_lon_lat.append([stri[0:idx1], x2, y2]) 
    
    
## Writing to an CSV file using Python
import csv

with open(foldername +'Bus_coordinates.csv', mode='w', newline= '') as file:
    writer = csv.writer(file)
    writer.writerow(['Bus_ID', 'Longitude', 'Latitude'])
    for i in range(0, len(bus_list)):
        writer.writerow([bus_list[i], lon_list[i], lat_list[i]])

In [205]:
# This constructs a graph to show the connection of the feeder
# The main files used are the series elements consisting of the lines and transformers


with open(foldername +'Lines.dss') as f:
    lines = f.readlines()

with open(foldername +'Transformers.dss') as f:
    trafo = f.readlines()
    
se = []
re = []

for i in range(0, len(lines), 2):
    stri = lines[i]

    if stri[4:8] == 'Line':
        idx0 = stri.index('enabled=')
        if stri[idx0:idx0+9] == 'enabled=y':
            idx1 = stri.index('bus1') #index of bus 1
            idx2 = stri.index('bus2') #index of bus 2
            idx3 = stri.index('switch') #index of switch
            bus1_name = stri[idx1+5:idx2-1]
            bus2_name = stri[idx2+5:idx3-1]
            se.append(bus1_name)
            re.append(bus2_name)    
            
            
# Extracting the transformers' primary and secondary buses
for i in range(0, len(trafo), 2):
    if trafo[i][0:25] == 'New Transformer.trans_Reg': #Regulator branches
        stri = trafo[i]
        idx1 = stri.index('buses') #index of bus 1
        idx2 = stri.index('(', idx1+1) #index of bus 2
        idx3 = stri.index(',', idx2+1) #index of winding
        idx4 = stri.index(')', idx3+1)  # index of winding
        bus1_name = stri[idx2+1:idx3-2]
        bus2_name = stri[idx3+1:idx4-2]
        se.append(bus1_name)
        re.append(bus2_name)
        
    else: 
        stri = trafo[i]
        idx1 = stri.index('bus') #index of bus 1
        idx2 = stri.index('bus', idx1+1) #index of bus 2
        idx3 = stri.index('wdg=2') #index of winding
        idx4 = stri.index('XHL')  # index of winding
        bus1_name = stri[idx1+4:idx3-1]
        bus2_name = stri[idx2+4:idx4-1]
        se.append(bus1_name)
        re.append(bus2_name)

In [206]:
# Checking for duplicate series elements (in lines and trafo)
news = []
for i in range(len(se)):
    print(se[i], re[i])
    temp = [se[i], re[i]]
    news.append(temp)

unique_elements = [] # empty list to hold unique elements from the list
dup_elements = [] # empty list to hold the duplicate elements from the list
for i in news:
    if i not in unique_elements:
        unique_elements.append(i)
    else:
        dup_elements.append(i) # this method catches the first duplicate entries, and appends them to the list
# The next step is to print the duplicate entries, and the unique entries
print("List of duplicates", dup_elements)
print("Unique Item List", unique_elements) # prints the final list of unique items


1076255_193_14121648_039.3 1076256_495_14121705_136.3
1076646_85_14121935_35.1.2.3 1076647_543_14122015_626.1.2.3
1076245_529_14121185_587.3 1076246_87_14121298_504.3
1076336_554_14120988_925.1.2.3 1076237_672_14120988_795.1.2.3
1076618_532_14121077_106.1.2.3 1076433_169_14120988_231.1.2.3
1075469_504_14121112_472.1.2.3 804377-240160-163153.1.2.3
1075419_55_14121146_514.1.2.3 1075469_504_14121112_472.1.2.3
1077107_769_14121192_744.1.2.3 1077048_353_14121194_37.1.2.3
1075414_746_14120775_579.1.2.3 1075409_404_14120685_756.1.2.3
1076241_728_14121080_683.3 1076245_529_14121185_587.3
1075208_905_14120779_557.1.2.3 1074988_853_14120784_716.1.2.3
1075677_625_14120995_999.1.2.3 1075508_408_14120999_327.1.2.3
1075419_55_14121146_514.1.2.3 1075415_061_14121155_45.1.2.3
1076619_774_14121203_126.1.2.3 1076620_068_14121213_122.1.2.3
1075423_501_14120999_002.1.2.3 1075419_55_14121146_514.1.2.3
1076641_657_14121567_268.1.2.3 1076640_589_14121643_75.1.2.3
1075423_501_14120999_002.1.2.3 1075413_529_14

In [207]:

# The goal is to plot the graph of all the buses
# Hence it is necessary to remove the phasing information i.e. '.1.2.3'
se_wo_phs = []
for s in se:
    length = len(s)

    if s[length - 12:] == '.1.2.3.1.2.3':
        n = length - 1
        replacement = ''
        res_str = s[0:n - 11] + replacement
        se_wo_phs.append(res_str)

    elif s[length - 12:] == '.1.2.3.3.1.2':
        n = length - 1
        replacement = ''
        res_str = s[0:n - 11] + replacement
        se_wo_phs.append(res_str)

    elif s[length - 12:] == '.1.2.3.2.1.3':
        n = length - 1
        replacement = ''
        res_str = s[0:n - 11] + replacement
        se_wo_phs.append(res_str)

    elif s[length - 12:] == '.3.1.2.1.2.3':
        n = length - 1
        replacement = ''
        res_str = s[0:n - 11] + replacement
        se_wo_phs.append(res_str)

    elif s[length - 12:] == '.2.1.3.1.2.3':
        n = length - 1
        replacement = ''
        res_str = s[0:n - 11] + replacement
        se_wo_phs.append(res_str)

    elif s[length - 6:] == '.1.2.3':
        n = length - 1
        replacement = ''
        res_str = s[0:n - 5] + replacement
        se_wo_phs.append(res_str)

    elif s[length - 6:] == '.1.3.2':
        n = length - 1
        replacement = ''
        res_str = s[0:n - 5] + replacement
        se_wo_phs.append(res_str)

    elif s[length - 6:] == '.3.2.1':
        n = length - 1
        replacement = ''
        res_str = s[0:n - 5] + replacement
        se_wo_phs.append(res_str)

    elif s[length - 6:] == '.3.1.2':
        n = length - 1
        replacement = ''
        res_str = s[0:n - 5] + replacement
        se_wo_phs.append(res_str)

    elif s[length - 6:] == '.2.3.1':
        n = length - 1
        replacement = ''
        res_str = s[0:n - 5] + replacement
        se_wo_phs.append(res_str)

    elif s[length - 6:] == '.2.1.3':
        n = length - 1
        replacement = ''
        res_str = s[0:n - 5] + replacement
        se_wo_phs.append(res_str)

    elif s[length - 4:] == '.1.1':
        n = length - 1
        replacement = ''
        res_str = s[0:n - 3] + replacement
        se_wo_phs.append(res_str)

    elif s[length - 4:] == '.1.3':
        n = length - 1
        replacement = ''
        res_str = s[0:n - 3] + replacement
        se_wo_phs.append(res_str)

    elif s[length - 4:] == '.1.2':
        n = length - 1
        replacement = ''
        res_str = s[0:n - 3] + replacement
        se_wo_phs.append(res_str)

    elif s[length - 4:] == '.2.1':
        n = length - 1
        replacement = ''
        res_str = s[0:n - 3] + replacement
        se_wo_phs.append(res_str)

    elif s[length - 4:] == '.2.2':
        n = length - 1
        replacement = ''
        res_str = s[0:n - 3] + replacement
        se_wo_phs.append(res_str)

    elif s[length - 4:] == '.2.3':
        n = length - 1
        replacement = ''
        res_str = s[0:n - 3] + replacement
        se_wo_phs.append(res_str)

    elif s[length - 4:] == '.3.1':
        n = length - 1
        replacement = ''
        res_str = s[0:n - 3] + replacement
        se_wo_phs.append(res_str)

    elif s[length - 4:] == '.3.2':
        n = length - 1
        replacement = ''
        res_str = s[0:n - 3] + replacement
        se_wo_phs.append(res_str)

    elif s[length - 4:] == '.3.3':
        n = length - 1
        replacement = ''
        res_str = s[0:n - 3] + replacement
        se_wo_phs.append(res_str)

    elif s[length - 2:] == '.1':
        n = length - 1
        replacement = ''
        res_str = s[0:n - 1] + replacement
        se_wo_phs.append(res_str)

    elif s[length - 2:] == '.2':
        n = length - 1
        replacement = ''
        res_str = s[0:n - 1] + replacement
        se_wo_phs.append(res_str)

    elif s[length - 2:] == '.3':
        n = length - 1
        replacement = ''
        res_str = s[0:n - 1] + replacement
        se_wo_phs.append(res_str)

    else:
        se_wo_phs.append(s)



re_wo_phs = []
for r in re:
    length = len(r)

    if r[length - 12:] == '.1.2.3.1.2.3':
        n = length - 1
        replacement = ''
        res_str = r[0:n - 11] + replacement
        re_wo_phs.append(res_str)

    elif r[length - 12:] == '.1.2.3.3.1.2':
        n = length - 1
        replacement = ''
        res_str = r[0:n - 11] + replacement
        re_wo_phs.append(res_str)

    elif r[length - 12:] == '.1.2.3.2.1.3':
        n = length - 1
        replacement = ''
        res_str = r[0:n - 11] + replacement
        re_wo_phs.append(res_str)

    elif r[length - 12:] == '.3.1.2.1.2.3':
        n = length - 1
        replacement = ''
        res_str = r[0:n - 11] + replacement
        re_wo_phs.append(res_str)

    elif r[length - 12:] == '.2.1.3.1.2.3':
        n = length - 1
        replacement = ''
        res_str = r[0:n - 11] + replacement
        re_wo_phs.append(res_str)

    elif r[length - 6:] == '.1.2.3':
        n = length - 1
        replacement = ''
        res_str = r[0:n - 5] + replacement
        re_wo_phs.append(res_str)

    elif r[length - 6:] == '.1.3.2':
        n = length - 1
        replacement = ''
        res_str = r[0:n - 5] + replacement
        re_wo_phs.append(res_str)

    elif r[length - 6:] == '.3.2.1':
        n = length - 1
        replacement = ''
        res_str = r[0:n - 5] + replacement
        re_wo_phs.append(res_str)

    elif r[length - 6:] == '.3.1.2':
        n = length - 1
        replacement = ''
        res_str = r[0:n - 5] + replacement
        re_wo_phs.append(res_str)


    elif r[length - 6:] == '.2.3.1':
        n = length - 1
        replacement = ''
        res_str = r[0:n - 5] + replacement
        re_wo_phs.append(res_str)

    elif r[length - 6:] == '.2.1.3':
        n = length - 1
        replacement = ''
        res_str = r[0:n - 5] + replacement
        re_wo_phs.append(res_str)

    elif r[length - 4:] == '.1.1':
        n = length - 1
        replacement = ''
        res_str = r[0:n - 3] + replacement
        re_wo_phs.append(res_str)

    elif r[length - 4:] == '.1.3':
        n = length - 1
        replacement = ''
        res_str = r[0:n - 3] + replacement
        re_wo_phs.append(res_str)

    elif r[length - 4:] == '.1.2':
        n = length - 1
        replacement = ''
        res_str = r[0:n - 3] + replacement
        re_wo_phs.append(res_str)

    elif r[length - 4:] == '.2.1':
        n = length - 1
        replacement = ''
        res_str = r[0:n - 3] + replacement
        re_wo_phs.append(res_str)

    elif r[length - 4:] == '.2.2':
        n = length - 1
        replacement = ''
        res_str = r[0:n - 3] + replacement
        re_wo_phs.append(res_str)

    elif r[length - 4:] == '.2.3':
        n = length - 1
        replacement = ''
        res_str = r[0:n - 3] + replacement
        re_wo_phs.append(res_str)

    elif r[length - 4:] == '.3.1':
        n = length - 1
        replacement = ''
        res_str = r[0:n - 3] + replacement
        re_wo_phs.append(res_str)

    elif r[length - 4:] == '.3.2':
        n = length - 1
        replacement = ''
        res_str = r[0:n - 3] + replacement
        re_wo_phs.append(res_str)

    elif r[length - 4:] == '.3.3':
        n = length - 1
        replacement = ''
        res_str = r[0:n - 3] + replacement
        re_wo_phs.append(res_str)

    elif r[length - 2:] == '.1':
        n = length - 1
        replacement = ''
        res_str = r[0:n - 1] + replacement
        re_wo_phs.append(res_str)

    elif r[length - 2:] == '.2':
        n = length - 1
        replacement = ''
        res_str = r[0:n - 1] + replacement
        re_wo_phs.append(res_str)

    elif r[length - 2:] == '.3':
        n = length - 1
        replacement = ''
        res_str = r[0:n - 1] + replacement
        re_wo_phs.append(res_str)

    else:
        re_wo_phs.append(r)

In [208]:
# Plotting thee graph
import networkx as nx
import matplotlib.pyplot as plt

g = nx.Graph()

new = []
for i in range(len(re_wo_phs)):
    g.add_edge(se_wo_phs[i], re_wo_phs[i])
#     print(se_wo_phs[i], re_wo_phs[i])
    temp = [se_wo_phs[i], re_wo_phs[i]]
    new.append(temp)
    
    
color_map = ['red' if node == '20101100104-XFO' else 'green' for node in g]
size_map = [50 if node == '20101100104-XFO' else 10 for node in g]
# nx.draw(g, with_labels = True)
#nx.draw(g, node_size=size_map, node_color=color_map)
#plt.show()

nx.is_connected(g)

True

In [209]:
## Writing to an CSV file using Python
import csv

with open(foldername +'Feeder_' + feedername+'_Node_connections_wo_norm_open_sw.csv', mode='w', newline= '') as file:
    writer = csv.writer(file)
    writer.writerow(['From_node', 'To_node'])
    for i in range(0, len(re_wo_phs)):
        writer.writerow([se_wo_phs[i], re_wo_phs[i]])

In [210]:
import pandas as pd

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

feeder = pd.read_csv(foldername+ "Bus_coordinates.csv")
#se_re = pd.read_csv("./Feeder_7130_Node_connections_wo_norm_open_sw.csv")
se_re = pd.read_csv( foldername + 'Feeder_' + feedername +'_Node_connections_wo_norm_open_sw.csv')



temp = list(feeder["Bus_ID"])
bus_ID_cap = []
for s in temp:
    caps = s.upper()
    bus_ID_cap.append(caps)

temp = list(se_re["From_node"])
se_cap = []
for s in temp:
    caps = s.upper()
    se_cap.append(caps)


temp = list(se_re["To_node"])
re_cap = []
for r in temp:
    caps = r.upper()
    re_cap.append(caps)

new_list=se_cap+re_cap

unique_elements = [] # empty list to hold unique elements from the list
dup_elements = [] # empty list to hold the duplicate elements from the list
for i in new_list:
    if i not in unique_elements:
        unique_elements.append(i)
    else:
        dup_elements.append(i)


busID_cap2 = []
bus_lon = []
bus_lat = []
for u in unique_elements:
    temp2 = bus_ID_cap.index(u)
    if temp2 >=0:
        lon = feeder["Longitude"][temp2]
        lat = feeder["Latitude"][temp2]
        busID_cap2.append(u)
        bus_lon.append(lon)
        bus_lat.append(lat)


## Writing to an CSV file using Python
import csv

with open(foldername+'Feeder_'+feedername+'_Bus_coordinates_no_sw.csv', mode='w', newline= '') as file:
    writer = csv.writer(file)
    writer.writerow(['Bus_ID', 'Longitude', 'Latitude'])
    for i in range(0, len(busID_cap2)):
        writer.writerow([busID_cap2[i], bus_lon[i], bus_lat[i]])







se_lon = []
se_lat = []
re_lon = []
re_lat = []
se_busID = []
re_busID = []


for s in se_cap:
    temp2 = busID_cap2.index(s)
    if temp2 >=0:
        lon = bus_lon[temp2]
        lat = bus_lat[temp2]
        se_busID.append(s)
        se_lon.append(lon)
        se_lat.append(lat)

for r in re_cap:
    temp2 = busID_cap2.index(r)
    if temp2 >=0:
        lon = bus_lon[temp2]
        lat = bus_lat[temp2]
        re_busID.append(r)
        re_lon.append(lon)
        re_lat.append(lat)


## Writing to an CSV file using Python
import csv

with open(foldername + 'se_re_lon_lat_nosw.csv', mode='w', newline= '') as file:
    writer = csv.writer(file)
    writer.writerow(['se_lon', 'se_lat', 're_lon', 're_lat', 'se_busID', 're_busID'])
    for i in range(0, len(se_lon)):
        writer.writerow([se_lon[i], se_lat[i], re_lon[i], re_lat[i], se_busID[i], re_busID[i]])


In [211]:
import pandas as pd
import numpy as np

feeder = pd.read_csv(foldername + "Feeder_"+feedername+"_Bus_coordinates_no_sw.csv")
se_re_lon_lat = pd.read_csv(foldername + "se_re_lon_lat_nosw.csv")


import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(go.Scattermapbox(lat=feeder["Latitude"], lon=feeder["Longitude"],
    mode = "markers", 
    marker = {'size': 10, 'color': "fuchsia",},
    hoverinfo = 'text',
    text = feeder["Bus_ID"],
                              ))

for i in range(len(se_re_lon_lat["se_lon"])):
    fig.add_trace(
        go.Scattermapbox(
            lon = [se_re_lon_lat["se_lon"][i], se_re_lon_lat["re_lon"][i]],
            lat = [se_re_lon_lat["se_lat"][i], se_re_lon_lat["re_lat"][i]],
            mode = 'lines',
            line = dict(width = 1, color = 'blue')
        ))
    

# getting center for plots:
lat_center = np.mean(feeder["Latitude"])
long_center = np.mean(feeder["Longitude"])

fig.update_layout(mapbox_style="open-street-map",
                  showlegend = False,
                  title_text = 'paths<br>Feeder on the map',)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.update_layout(mapbox = {'zoom': 10, 'center': {'lat': lat_center, 
                          'lon': long_center},}, height=700)

fig.show()

In [212]:
[len(c) for c in sorted(nx.connected_components(g), key=len, reverse=True)]

[94]